# Patient Data Export

A notebook exporting patient data for study analyses.

## Definitions

### Imports

In [ ]:
import dataclasses
import io
import itertools
import json
import operator
import os
import pathlib
import pprint
import re
from enum import Enum
from typing import Dict, List, Optional, Union

import bson.objectid
import IPython.display
import ipywidgets
import nbformat
import pandas as pd
import pytz
import pyzipper
import scope.database.date_utils as date_utils
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE
from scope.documents import document_set
from scope.populate.data.archive import Archive

### Constants and Flags

In [ ]:
# In development, it can be helpful to sample a subset of patients.
# If DEVELOPMENT_SAMPLE_PATIENTS <= 0, process all patients.
# If DEVELOPMENT_SAMPLE_PATIENTS > 0, randomly sample DEVELOPMENT_SAMPLE_PATIENTS patients.
DEVELOPMENT_SAMPLE_PATIENTS: int = -1

# In development, it can be helpful to skip per-patient export.
# If DEVELOPMENT_EXPORT_PER_PATIENT_DOCUMENTS, include per-patient export.
DEVELOPMENT_EXPORT_PER_PATIENT_DOCUMENTS: bool = False

# In development, it can be helpful to skip documents export.
# If DEVELOPMENT_EXPORT_COMBINED_DOCUMENTS, include documents export.
DEVELOPMENT_EXPORT_COMBINED_DOCUMENTS: bool = False

### Utilities

### Utility: documentation_as_markdown

Returns a string containing markdown content recovered from a cell in this notebook.

Intended to allow including the content of markdown cells as documentation in an export.

In [ ]:
def documentation_as_markdown(documentation_name: str) -> str:
    # Load this same notebook.
    notebook = nbformat.read("patientdata.ipynb", nbformat.NO_CONVERT)

    # Go through each cell, looking for a match.
    for cell_current in notebook["cells"]:
        match = True
        if match:
            match = cell_current["cell_type"] == "markdown"
        if match:
            match = re.match(
                "^(#*) Documentation: ({})\\n(.*)".format(documentation_name),
                cell_current["source"],
            )

        if match:
            return cell_current["source"]

    # If no match was found, raise a ValueError.
    raise ValueError(
        "No matching documentation cell found: {}".format(documentation_name)
    )

### Utility: ExportFile

The path and contents of a file to be exported.

In [ ]:
class ExportFileType(Enum):
    BYTES = "BYTES"
    CSV = "CSV"
    EXCEL = "EXCEL"
    MARKDOWN = "MARKDOWN"


@dataclasses.dataclass(frozen=True)
class ExportFile:
    path: pathlib.Path
    type: ExportFileType
    bytes: Optional[bytes]
    text: Optional[str]

### Utility: export_dataframe_as_csv

In [ ]:
def export_dataframe_as_csv(
    path: pathlib.Path,
    df: pd.DataFrame,
):
    path = path.with_suffix(path.suffix + ".csv")

    csv_text = df.to_csv(index=False)

    export_file_list.append(
        ExportFile(
            path=path,
            type=ExportFileType.CSV,
            bytes=None,
            text=csv_text,
        )
    )

### Utility: export_dataframe_as_excel

In [ ]:
def export_dataframe_as_excel(
    path: pathlib.Path,
    df: pd.DataFrame,
):
    path = path.with_suffix(path.suffix + ".xlsx")

    iobytes = io.BytesIO()
    df.to_excel(iobytes, index=False)
    excel_bytes = iobytes.getvalue()

    export_file_list.append(
        ExportFile(
            path=path,
            type=ExportFileType.EXCEL,
            bytes=excel_bytes,
            text=None,
        )
    )

### Utility: export_dataframe

In [ ]:
def export_dataframe(
    path: pathlib.Path,
    df: pd.DataFrame,
):
    export_dataframe_as_csv(path, df)
    export_dataframe_as_excel(path, df)

### Utility: export_file_bytes

In [ ]:
def export_file_bytes(
    path: pathlib.Path,
    file_bytes: bytes,
):
    export_file_list.append(
        ExportFile(
            path=path,
            type=ExportFileType.BYTES,
            bytes=file_bytes,
            text=None,
        )
    )

### Utility: export_markdown

In [ ]:
def export_markdown(
    path: pathlib.Path,
    markdown: str,
):
    path = path.with_suffix(path.suffix + ".md")

    export_file_list.append(
        ExportFile(
            path=path,
            type=ExportFileType.MARKDOWN,
            bytes=None,
            text=markdown,
        )
    )

### Utility: dataframe_sanitize

Sanitize contents of a dataframe that otherwise cannot be written to Excel.

In [ ]:
def dataframe_sanitize(df: pd.DataFrame) -> pd.DataFrame:
    def sanitize_cell(value):
        if type(value) == str:
            value = ILLEGAL_CHARACTERS_RE.sub("?", value)

        return value

    return df.map(sanitize_cell)

### Utility: dataframe_format_export

Formats a dataframe for export.

In [ ]:
def dataframe_format_export(
    df: pd.DataFrame,
    *,
    drop_empty_columns: Optional[bool] = False,
    drop_columns: Optional[List[str]] = None,
    rename_columns: Optional[Dict[str, str]] = None,
    sort_columns: Optional[List[str]] = None,
    sort_rows_by_columns: Optional[List[str]] = None,
) -> pd.DataFrame:
    # Ensure we are modifying a copy.
    df = df.copy()

    # If requested, drop empty columns.
    if drop_empty_columns:
        empty_columns = []
        for column_current in df.columns:
            if (
                df[column_current].isnull().all()
                or (
                    df[column_current].astype(str).str.strip().isin(["", "nan", "None"])
                ).all()
            ):
                empty_columns.append(column_current)

        df = df.drop(columns=empty_columns)

    # If requested, drop specific columns.
    # Be robust to the possibility that a column is not present.
    if drop_columns:
        df = df.drop(columns=drop_columns, errors="ignore")

    # If requested, rename specific columns.
    if rename_columns:
        df = df.rename(columns=rename_columns)

    # If requested, sort specific columns to the front.
    # Be robust to the possibility that a column is not present.
    # Be robust to the presence of additional columns.
    # Preserve existing order of additional columns after requested columns.
    if sort_columns:
        sort_columns = [
            column_current
            for column_current in sort_columns
            if column_current in df.columns
        ]
        sort_columns = sort_columns + [
            column_current
            for column_current in df.columns
            if column_current not in sort_columns
        ]

        df = df[sort_columns]

    # If requested, sort rows by specific columns.
    # Be robust to the possibility that a column is not present.
    if sort_rows_by_columns:
        sort_rows_by_columns = [
            column_current
            for column_current in sort_rows_by_columns
            if column_current in df.columns
        ]

        df = df.sort_values(sort_rows_by_columns)

    return df

## Input

### Utility: archive_dir_path

In [ ]:
# Path containing encrypted archives.
archive_dir_path = "../../../secrets/data"

### Input Archive Suffix: archive_suffix 

In [ ]:
# Obtain suffix indicating desired version of encrypted archives.
# Do not include the '.zip' suffix.
def input_archive_suffix():
    # Based on archives in our path, identify possible suffixes.
    pattern = re.compile("archive_(multicare|scca)_([^_]+)_(\\d{8})(?:_(.+))?.zip")
    archive_file_names = [
        name_current
        for name_current in os.listdir(archive_dir_path)
        if pattern.search(name_current)
    ]
    archive_matches = [
        re.match(pattern, name_current) for name_current in archive_file_names
    ]
    archive_suffixes = list(
        {
            "{}_{}{}".format(
                match_current.group(2),
                match_current.group(3),
                "_" + match_current.group(4) if len(match_current.groups()) > 3 else "",
            )
            for match_current in archive_matches
        }
    )

    # If there is only one possible value, no need for a choice.
    archive_suffix = None
    if len(archive_suffixes) == 1:
        archive_suffix = archive_suffixes[0]
    else:
        for id_current, suffix_current in enumerate(archive_suffixes, 1):
            print("[{}]: {}".format(id_current, suffix_current))

        archive_suffix = archive_suffixes[
            int(input("Encrypted archive suffix index: ")) - 1
        ]

    print(archive_suffix)

    return archive_suffix


archive_suffix = input_archive_suffix()

### Input Archive Password: archive_password

In [ ]:
# Obtain password to encrypted archives.
archive_password = input("Encrypted archive password: ")

## Load Data

### Decrypt Archives

#### Data: archive_multicare

In [ ]:
def decrypt_archive_multicare():
    # Obtain name for each archive.
    archive_multicare_file_name = "archive_multicare_{}.zip".format(archive_suffix)

    # Obtain a full path to encrypted archive, relative to the location of the notebook.
    # Expects the encrypted archive to be in the "secrets/data" directory.
    archive_multicare_path = pathlib.Path(
        archive_dir_path,
        archive_multicare_file_name,
    )

    print("Decrypting archive:")
    print("{}".format(archive_multicare_path.resolve()))

    # Obtain the archive.
    archive_multicare = Archive.read_archive(
        archive_path=archive_multicare_path,
        password=archive_password,
    )

    print("{} documents.".format(len(archive_multicare.entries.values())))

    return archive_multicare


archive_multicare = decrypt_archive_multicare()

#### Data: archive_scca

In [ ]:
def decrypt_archive_scca():
    # Obtain name for each archive.
    archive_scca_file_name = "archive_scca_{}.zip".format(archive_suffix)

    # Obtain a full path to encrypted archive, relative to the location of the notebook.
    # Expects the encrypted archive to be in the "secrets/data" directory.
    archive_scca_path = pathlib.Path(
        archive_dir_path,
        archive_scca_file_name,
    )

    print("Decrypting archive:")
    print("{}".format(archive_scca_path.resolve()))

    # Obtain the archive.
    archive_scca = Archive.read_archive(
        archive_path=archive_scca_path,
        password=archive_password,
    )

    print("{} documents.".format(len(archive_scca.entries.values())))

    return archive_scca


archive_scca = decrypt_archive_scca()

### Decrypt MRN to RecordId

#### Data: mrn_to_record_id_bytes

#### Data: mrn_to_record_id

In [ ]:
def decrypt_mrn_to_record_id():
    # Obtain a full path to encrypted archive, relative to the location of the notebook.
    # Expects the encrypted archive to be in the "secrets/data" directory.
    archive_mrn_to_record_id_path = pathlib.Path(
        archive_dir_path,
        "archive_mrn_to_record_id.zip",
    )

    # Open the file
    with open(
        archive_mrn_to_record_id_path,
        mode="rb",
    ) as archive_file:
        with pyzipper.AESZipFile(
            archive_file,
            "r",
            compression=pyzipper.ZIP_LZMA,
            encryption=pyzipper.WZ_AES,
        ) as archive_zipfile:
            # Set the zipfile password
            archive_zipfile.setpassword(archive_password.encode("utf-8"))

            # Confirm the zipfile is valid
            if archive_zipfile.testzip():
                raise ValueError("Invalid archive or password")

            # Retrieve the Excel file.
            excel_bytes = archive_zipfile.read(
                "archive_mrn_to_record_id/archive_mrn_to_record_id.xlsx"
            )
            df_mrn_to_record_id = pd.read_excel(io.BytesIO(excel_bytes))

            # Ensure MRN are treated as strings.
            df_mrn_to_record_id["MRN"] = df_mrn_to_record_id["MRN"].astype(str)

            # Return the Excel file and a dictionary.
            return (
                excel_bytes,
                df_mrn_to_record_id.set_index("MRN")["recordId"].to_dict(),
            )


mrn_to_record_id_bytes, mrn_to_record_id = decrypt_mrn_to_record_id()

## Process Archives

### Combine Patients DataFrames

In [ ]:
def combine_patients_dataframes():
    # Get patient documents from MultiCare.
    documents_multicare_patients = (
        archive_multicare.collection_documents(
            collection="patients",
        )
        .remove_sentinel()
        .remove_revisions()
    )
    df_multicare_patients = pd.DataFrame.from_records(
        documents_multicare_patients.documents
    )
    df_multicare_patients["database"] = "multicare"

    # Get patient documents from SCCA.
    documents_scca_patients = (
        archive_scca.collection_documents(
            collection="patients",
        )
        .remove_sentinel()
        .remove_revisions()
    )
    df_scca_patients = pd.DataFrame.from_records(documents_scca_patients.documents)
    df_scca_patients["database"] = "fhcc"

    # Unify all current patient documents.
    df_combined_patients = pd.concat(
        [df_multicare_patients, df_scca_patients]
    ).reset_index(drop=True)

    # Sanitize once so contents dataframe can be exported.
    df_combined_patients = dataframe_sanitize(df_combined_patients)

    return df_combined_patients


df_archive_patients_raw = combine_patients_dataframes()

### Reset Filtering and Sampling

#### Data: df_archive_patients

In [ ]:
df_archive_patients = df_archive_patients_raw.copy()

### Filter Pilot Patients

Remove the 6 pilot patients.

In [ ]:
df_archive_patients = df_archive_patients.drop(
    df_archive_patients[
        df_archive_patients["patientId"].isin(
            [
                "ymzwx6e6w6kqi",
                "mmmb54v52l7re",
                "ouoa4ucldbhie",
                "zazst4yu23a5q",
                "wf4btxqjtd2oa",
                "s3bcmgmp7gdss",
            ]
        )
    ].index
).reset_index(drop=True)

### Sample Patients

In development, it can be helpful to sample a subset of patients.

In [ ]:
if DEVELOPMENT_SAMPLE_PATIENTS > 0:
    df_archive_patients = df_archive_patients.sample(
        n=DEVELOPMENT_SAMPLE_PATIENTS
    ).reset_index(drop=True)

### Utility: patient_documents

In [ ]:
# Create a helper for accessing the document collection of a unified patient.
def patient_documents(row_patient) -> document_set.DocumentSet:
    if row_patient["database"] == "multicare":
        archive = archive_multicare
    elif row_patient["database"] == "fhcc":
        archive = archive_scca
    else:
        raise ValueError()

    return archive.collection_documents(collection=row_patient["collection"])

## Prepare Data

### Prepare Patients

#### Documentation: Patients

All patients included in this export, based on `patientIdentity` documents.
These are stored separately from the collection of documents associated with each patient.
They provide an index of all patients, then we can access the collection of documents for each individual patient.

A `patientIdentity` document may have been modified throughout the study (e.g., to change a patient name or email address).
If the document was modified, this export includes only the final version.
There is therefore exactly one row per patient.

Data is originally taken from two database exports: from FHCC and from MultiCare.
Raw data are merged, then a `database` column is added to indicate the origin of each patient.

There were 6 pilot patients. These have been completely removed.

TODO: The patient with `recordId` `1747` maps to two different `MRN` and `patientId`.
This was because they moved from one system to another during the study.
There is still a need to decide how to special case that patient.

#### Data: df_patients_raw

In [ ]:
df_patients_raw = df_archive_patients.copy()

#### Data: df_patients

Remove most fields, so they are not needlessly visible during export script development.

Use `MRN` to map each patient to a `recordId`.

In [ ]:
df_patients = df_patients_raw.copy()

# Ensure MRN are treated as strings.
df_patients["MRN"] = df_patients["MRN"].astype(str)

# Apply a transform to look up the recordId.
def transform_add_record_id(
    df_patients: pd.DataFrame,
) -> pd.DataFrame:
    def _transform_add_record_id_from_mrn(row):
        return str(mrn_to_record_id.get(row["MRN"], "???"))

    df_patients = df_patients.copy()
    df_patients["recordId"] = df_patients.apply(
        _transform_add_record_id_from_mrn, axis=1
    )

    return df_patients


df_patients = transform_add_record_id(df_patients)

# "collection" cannot be removed at this point, as it is used by later processing.
df_patients = dataframe_format_export(
    df_patients,
    drop_columns=[
        # Remove clutter.
        "_id",
        "_rev",
        "_set_id",
        "_type",
        # Remove identifiers.
        "cognitoAccount",  # Includes email.
        "name",
    ],
    sort_columns=[
        "recordId",
        "database",
        "patientId",
        "MRN",
        "collection",
    ],
    sort_rows_by_columns=[
        "recordId",
        "database",
        "patientId",
    ],
)

#### Data: patient_id_to_record_id

In [ ]:
patient_id_to_record_id = (
    df_patients.copy().set_index("patientId")["recordId"].to_dict()
)

### Prepare Per-Patient DocumentSets and DataFrames

#### Data: patient_id_to_documentset

In [ ]:
def prepare_patient_id_to_documentset():
    patient_id_to_documentset = {}

    progress_max = len(df_patients)
    progress_patient_count = ipywidgets.IntProgress(min=0, max=progress_max)

    print("Preparing DocumentSet for {} Patients".format(progress_max))
    IPython.display.display(progress_patient_count)

    progress_patient_count.description = "{}/{}".format(0, progress_max)
    for patient_count, (row_current, patient_current) in enumerate(
        df_patients.iterrows()
    ):
        patient_id_current = patient_current["patientId"]
        patient_collection = patient_documents(patient_current.to_dict())

        patient_id_to_documentset[patient_id_current] = patient_collection

        progress_patient_count.description = "{}/{}".format(
            patient_count + 1, progress_max
        )
        progress_patient_count.value = patient_count + 1

    return patient_id_to_documentset


patient_id_to_documentset = prepare_patient_id_to_documentset()

#### Transform: transform_add_created

Add a `_created` column to each document.

In [ ]:
def transform_add_created(
    df_documents: pd.DataFrame,
) -> pd.DataFrame:
    def _transform_add_created_from_id(row):
        datetime_parsed = bson.objectid.ObjectId(row["_id"]).generation_time.astimezone(
            pytz.utc
        )
        datetime_pacific = datetime_parsed.astimezone(
            pytz.timezone("America/Los_Angeles")
        )

        return datetime_pacific.strftime("%Y-%m-%dT%H:%M:%S")

    df_documents = df_documents.copy()
    df_documents["_created"] = df_documents.apply(
        _transform_add_created_from_id, axis=1
    )

    return df_documents

#### Transform: transform_add_record_id_and_patient_id

Add a `patientId` column to each document.

In [ ]:
def transform_add_record_id_and_patient_id(
    df_documents: pd.DataFrame,
    *,
    patient_id,
) -> pd.DataFrame:
    df_documents = df_documents.copy()
    df_documents["recordId"] = patient_id_to_record_id[patient_id]
    df_documents["_patientId"] = patient_id

    return df_documents

#### Data: patient_id_to_df_documents_raw

In [ ]:
def prepare_patient_id_to_df_documents_raw():
    patient_id_to_df_documents_raw = {}

    progress_max = len(patient_id_to_documentset)
    progress_patient_count = ipywidgets.IntProgress(min=0, max=progress_max)

    print("Preparing DataFrame for {} Patients".format(progress_max))
    IPython.display.display(progress_patient_count)

    progress_patient_count.description = "{}/{}".format(0, progress_max)
    for patient_count, (patient_id_current, patient_documentset_current) in enumerate(
        patient_id_to_documentset.items()
    ):
        df_documents_raw_current = pd.DataFrame.from_records(
            patient_documentset_current.documents
        )

        # Sanitize contents for export.
        df_documents_raw_current = dataframe_sanitize(df_documents_raw_current)

        # Apply minimal transforms to the "raw" documents.
        df_documents_raw_current = transform_add_created(
            df_documents_raw_current,
        )
        df_documents_raw_current = transform_add_record_id_and_patient_id(
            df_documents_raw_current, patient_id=patient_id_current
        )

        patient_id_to_df_documents_raw[patient_id_current] = df_documents_raw_current

        progress_patient_count.description = "{}/{}".format(
            patient_count + 1, progress_max
        )
        progress_patient_count.value = patient_count + 1

    return patient_id_to_df_documents_raw


patient_id_to_df_documents_raw = prepare_patient_id_to_df_documents_raw()

In [ ]:
df_documents_raw = pd.concat(patient_id_to_df_documents_raw.values(), ignore_index=True)

## Transform Documents

### Documentation: Common Fields

Several common fields are shared across different analysis exports:

- `recordId` identifies the patient.

- `_patientId` and `_docId` uniquely identify a document.
  These are intended primarily for inspection and data cleaning, and are not intended to be used in analyses.

- If there could be multiple instances of a document type (e.g., multiple assessment logs, multiple mood logs),
  then documents include a type-specific identifier (e.g., `_assessmentLogId`, `_moodLogId`).

- `_rev` identifies the version of the document (i.e., the revision).
  If only one instance of a document type is allowed (e.g., a single safety plan),
  this identifies revisions of that document over time.
  If there could be multiple instances of a document type,
  the combination of the type-specific identifier and the version identifies revisions of specific instances.

- `_created` indicates when a document was created, recovered from an encoding within `_docId`.
  This is provided in Pacific time.

### Documentation: Assessments

Exported from `assessmentLog` documents. A single row is included for each `assessmentLog`.

Includes common fields documented in `commonFields.md`.

Notable values:

- `assessmentId` will be either `gad-7` or `phq-9`.

- Documents include both `_created` and `recordedDate`.

  - Per commond fields, `_created` indicates when a document was created.

  - `recordedDate` indicates what date was indicated for an `assessmentLog`.

  - For assessment logs submitted via the patient app, these were the same.

  - For assessment logs submitted via the provider registry, the provider entered the date of the assessment log.

Notable transformations:

- `recordedDate` is calculated in Pacific time from a raw `recordedDateTime`.

- `submittedBy` is calculated from a raw `patientSubmitted`.

- If individual scale components were available in a raw `pointValues`,
  they were promoted to columns (e.g., `gad7Anxious`, `phq9Interest`).

- An assessment score (i.e., `gad7Score`, `phq9Score`)
  was taken from a raw `totalScore` or by summing the values in a raw `pointValues`.

TODO: Inspecting a sample of revised assessment logs suggests there will not be a simple approach to data cleaning.

TODO: `submittedBy` value of `SW` is potentially misleading.
These were created using the registry, but not necessarily by a social worker.

TODO: `todo_scheduledAssessmentId` is present for patient-submitted entries, may allow recovering information about an assessment schedule.
It is unclear how correct this field is, so it may be best to not rely upon it.

TODO: `todo_submittedByProviderId` is present for registry-submitted entries, may allow recovering information about who submitted an entry.

### Transform: transform_assessment_log

In [ ]:
def transform_assessment_log(
    df_documents: pd.DataFrame,
) -> pd.DataFrame:
    # Pull each value of the gad-7 assessment scale out to its own column.
    def _transform_gad7_points(
        df_documents: pd.DataFrame,
    ) -> pd.DataFrame:
        def _factory_transform_gad7_points_key(keyJson):
            def _transform_gad7_points_key(row):
                if row["_type"] != "assessmentLog":
                    return None
                if row["assessmentId"] != "gad-7":
                    return None
                if not row["pointValues"]:
                    return None

                return row["pointValues"][keyJson]

            return _transform_gad7_points_key

        gad7EnumMap = {
            "Anxious": "gad7Anxious",
            "Constant worrying": "gad7ConstantWorrying",
            "Worrying too much": "gad7WorryingTooMuch",
            "Trouble relaxing": "gad7TroubleRelaxing",
            "Restless": "gad7Restless",
            "Irritable": "gad7Irritable",
            "Afraid": "gad7Afraid",
        }

        for (keyJson, keyExport) in gad7EnumMap.items():
            df_documents[keyExport] = df_documents.apply(
                _factory_transform_gad7_points_key(keyJson), axis=1
            )

        return df_documents

    # Obtain or calculate a gad-7 score.
    def _transform_gad7_score(row):
        if row["_type"] != "assessmentLog":
            return None
        if row["assessmentId"] != "gad-7":
            return None

        # Some rows already provide a totalScore.
        if "totalScore" in row and not pd.isna(row["totalScore"]):
            return row["totalScore"]

        # Otherwise we need to sum the pointValues.
        if "pointValues" in row and row["pointValues"]:
            return sum(row["pointValues"].values())

        # We should always have one or the other.
        raise ValueError()

    # Pull each value of the phq-9 assessment scale out to its own column.
    def _transform_phq9_points(
        df_documents: pd.DataFrame,
    ) -> pd.DataFrame:
        def _factory_transform_phq9_points_key(keyJson):
            def _transform_phq9_points_key(row):
                if row["_type"] != "assessmentLog":
                    return None
                if row["assessmentId"] != "phq-9":
                    return None
                if not row["pointValues"]:
                    return None

                return row["pointValues"][keyJson]

            return _transform_phq9_points_key

        phq9EnumMap = {
            "Interest": "phq9Interest",
            "Mood": "phq9Mood",
            "Sleep": "phq9Sleep",
            "Energy": "phq9Energy",
            "Appetite": "phq9Appetite",
            "Guilt": "phq9Guilt",
            "Concentrating": "phq9Concentrating",
            "Motor": "phq9Motor",
            "Suicide": "phq9Suicide",
        }

        for (keyJson, keyExport) in phq9EnumMap.items():
            df_documents[keyExport] = df_documents.apply(
                _factory_transform_phq9_points_key(keyJson), axis=1
            )

        return df_documents

    # Obtain or calculate a phq-9 score.
    def _transform_phq9_score(row):
        if row["_type"] != "assessmentLog":
            return None
        if row["assessmentId"] != "phq-9":
            return None

        # Some rows already provide a totalScore.
        if "totalScore" in row and not pd.isna(row["totalScore"]):
            return row["totalScore"]

        # Otherwise we need to sum the pointValues.
        if "pointValues" in row and row["pointValues"]:
            return sum(row["pointValues"].values())

        # We should always have one or the other.
        raise ValueError()

    # Format a recordedDate.
    def _transform_recorded_date(row):
        if row["_type"] != "assessmentLog":
            return None

        datetime_parsed = date_utils.parse_datetime(datetime=row["recordedDateTime"])
        datetime_pacific = datetime_parsed.astimezone(
            pytz.timezone("America/Los_Angeles")
        )

        return datetime_pacific.strftime("%Y-%m-%d")

    def _transform_scheduled_assessment_id(row):
        if row["_type"] != "assessmentLog":
            return None

        # Logs labeled on-demand should all be submitted by a social worker.
        if row["scheduledAssessmentId"] == "on-demand":
            if row["patientSubmitted"] == True:
                raise ValueError()

            return None

        return row["scheduledAssessmentId"]

    # Format a submittedBy column as requested.
    def _transform_submitted_by(row):
        if row["_type"] != "assessmentLog":
            return None

        if row["patientSubmitted"] == True:
            return "Pt"
        elif row["patientSubmitted"] == False:
            return "SW"
        else:
            raise ValueError()

    df_documents = df_documents.copy()
    df_documents = _transform_gad7_points(df_documents)
    df_documents["gad7Score"] = df_documents.apply(_transform_gad7_score, axis=1)
    df_documents = _transform_phq9_points(df_documents)
    df_documents["phq9Score"] = df_documents.apply(_transform_phq9_score, axis=1)
    df_documents["assessmentLogRecordedDate"] = df_documents.apply(
        _transform_recorded_date, axis=1
    )
    df_documents["assessmentLogScheduledAssessmentId"] = df_documents.apply(
        _transform_scheduled_assessment_id, axis=1
    )
    df_documents["assessmentLogSubmittedBy"] = df_documents.apply(
        _transform_submitted_by, axis=1
    )

    return df_documents

### Documentation: Mood Logs

Exported from `moodLog` documents. A single row is included for each `moodLog`.

Includes common fields documented in `commonFields.md`.

Notable values:

- All values of `_rev` are `1`, as it was not possible to edit a mood log.

### Transform: transform_mood_log

In [ ]:
def transform_mood_log(
    df_documents: pd.DataFrame,
) -> pd.DataFrame:
    # No transformations are needed.
    return df_documents

### Utility: apply_transforms

In [ ]:
def apply_transforms(
    df_documents: pd.DataFrame,
) -> pd.DataFrame:
    df_documents = transform_assessment_log(
        df_documents,
    )
    df_documents = transform_mood_log(
        df_documents,
    )

    return df_documents

### Data: patient_id_to_df_documents

In [ ]:
def prepare_transform_patient_documents():
    patient_id_to_df_documents = {}

    progress_max = len(patient_id_to_df_documents_raw)
    progress_patient_count = ipywidgets.IntProgress(min=0, max=progress_max)

    print("Transforming DataFrame for {} Patients".format(progress_max))
    IPython.display.display(progress_patient_count)

    progress_patient_count.description = "{}/{}".format(0, progress_max)
    for patient_count, (patient_id_current, df_documents_raw_current) in enumerate(
        patient_id_to_df_documents_raw.items()
    ):
        patient_id_to_df_documents[patient_id_current] = apply_transforms(
            df_documents_raw_current.copy(),
        )

        progress_patient_count.description = "{}/{}".format(
            patient_count + 1, progress_max
        )
        progress_patient_count.value = patient_count + 1

    return patient_id_to_df_documents


patient_id_to_df_documents = prepare_transform_patient_documents()

### Prepare Combined Documents DataFrame

#### Data: df_documents_raw

In [ ]:
df_documents_raw = pd.concat(patient_id_to_df_documents_raw.values(), ignore_index=True)

#### Data: df_documents

In [ ]:
df_documents = pd.concat(patient_id_to_df_documents.values(), ignore_index=True)

## Export

### Reset Export File List

In [ ]:
export_file_list: List[ExportFile] = []

### Documentation: Export

This folder contains an export of SCOPE platform data.
This includes patient data, should always be stored in an encrypted format, and should be treated as highly sensitive.
Tables are commonly exported in both Excel and comma-separated formats
(i.e., `.xlsx` via `pd.DataFrame.to_excel`, `.csv` via `pd.DataFrame.to_csv`).
Excel will typically be easier to visually inspect,
while comma-separated may be easier for R-based analyses.

The root folder contains exports that are intended to be used in analyses.

- `patients` is a list of all patients included in the export. Documentation in `patients.md`.

- `assessments.gad7` is an export of all GAD-7 assessments. Documentation in `assessments.md`.
- `assessments.phq9` is an export of all PHQ-9 assessments. Documentation in `assessments.md`.

- `moodLogs.phq9` is an export of all mood logs. Documentation in `moodLogs.md`.

The `data` folder contains additional exports, intended to support inspection of and communication around underlying data.
Exports are prepared in a multi-step process, converting a database of JSON documents into tables for analyses.

- Note that document tables are sparse, as most documents (i.e., rows) do not contain most values (i.e., columns).
  Each row includes a `_type` taken from the JSON document, commonly used for filtering to specific types of documents.

- Raw JSON documents are first loaded into a table for each patient.
  This is the raw underlying data, included in the export to allow inspection.
  The `data/patients` folder includes a folder for each patient, named according to the `patientId`.  

- Tables for all patients are combined in a single large table, exported as `documents.raw`.
  This is all of the raw data and contains everything that is available.

- A series of transformations are applied to the single large table.
  Each transformation computes one or more new columns that are added to the single large table.

- After all transformations are completed, the single large table is exported as `documents.transformed`.
  Filtering the single large table by `patientId`, transformed tables are also exported to the each folder in `data/patients`.
  These are included in the export to allow inspection.

- Tables intended for analyses are then exported as different subsets of `documents.transformed`.

The `config` folder contains additional configuration that was used in the export.
If a change in configuration is needed, these files should be modified and then used in a new export.

- `archive_mrn_to_record_id.xlsx` contains the mapping from `MRN` to `recordId` that was used in this export.


In [ ]:
export_markdown(pathlib.Path("documentation"), documentation_as_markdown("Export"))
export_markdown(
    pathlib.Path("commonFields"), documentation_as_markdown("Common Fields")
)

### Patients

- Documented above in "Documentation: Patients Export".

In [ ]:
export_markdown(
    pathlib.Path("patients"),
    documentation_as_markdown("Patients"),
)

export_file_bytes(
    pathlib.Path(
        "config",
        "archive_mrn_to_record_id.xlsx",
    ),
    mrn_to_record_id_bytes,
)

export_dataframe(
    pathlib.Path(
        "data",
        "patients.raw",
    ),
    df_patients_raw,
)

export_dataframe(
    pathlib.Path(
        "patients",
    ),
    dataframe_format_export(
        df_patients,
        drop_columns=["collection"],
    ),
)

### Per-Patient Documents

In [ ]:
def export_per_patient_documents():
    progress_max = len(df_patients)
    progress_patient_count = ipywidgets.IntProgress(min=0, max=progress_max)

    print("Per-Patient Export for {} Patients".format(progress_max))
    IPython.display.display(progress_patient_count)

    progress_patient_count.description = "{}/{}".format(0, progress_max)
    for patient_count, (row_current, patient_current) in enumerate(
        df_patients.iterrows()
    ):
        patient_id_current = patient_current["patientId"]

        df_documents_raw_current = patient_id_to_df_documents_raw[patient_id_current]
        export_dataframe(
            pathlib.Path(
                "data",
                "patients",
                "patient_{}".format(patient_id_current),
                "patient_{}.raw".format(patient_id_current),
            ),
            df_documents_raw_current,
        )

        df_documents_current = patient_id_to_df_documents[patient_id_current]
        export_dataframe(
            pathlib.Path(
                "data",
                "patients",
                "patient_{}".format(patient_id_current),
                "patient_{}.transformed".format(patient_id_current),
            ),
            df_documents_current,
        )

        progress_patient_count.description = "{}/{}".format(
            patient_count + 1, progress_max
        )
        progress_patient_count.value = patient_count + 1


if DEVELOPMENT_EXPORT_PER_PATIENT_DOCUMENTS:
    export_per_patient_documents()

### Combined Documents

In [ ]:
if DEVELOPMENT_EXPORT_COMBINED_DOCUMENTS:
    export_dataframe(
        pathlib.Path(
            "data",
            "documents.raw",
        ),
        df_documents_raw,
    )

    export_dataframe(
        pathlib.Path(
            "data",
            "documents.transformed",
        ),
        df_documents,
    )

### Analysis: Assessments

In [ ]:
def export_analysis_assessments():
    # Documentation of this analysis.
    export_markdown(
        pathlib.Path("assessments"),
        documentation_as_markdown("Assessments"),
    )

    # Preliminary GAD-7 documents.
    export_dataframe(
        pathlib.Path(
            "data",
            "assessments.gad7.raw",
        ),
        dataframe_format_export(
            df_documents_raw[
                (df_documents_raw["_type"] == "assessmentLog")
                & (df_documents_raw["assessmentId"] == "gad-7")
            ],
            drop_empty_columns=True,
        ),
    )

    export_dataframe(
        pathlib.Path(
            "data",
            "assessments.gad7.transformed",
        ),
        dataframe_format_export(
            df_documents[
                (df_documents["_type"] == "assessmentLog")
                & (df_documents["assessmentId"] == "gad-7")
            ],
            drop_empty_columns=True,
        ),
    )

    # Preliminary PHQ-9 documents.
    export_dataframe(
        pathlib.Path(
            "data",
            "assessments.phq9.raw",
        ),
        dataframe_format_export(
            df_documents_raw[
                (df_documents_raw["_type"] == "assessmentLog")
                & (df_documents_raw["assessmentId"] == "phq-9")
            ],
            drop_empty_columns=True,
        ),
    )

    export_dataframe(
        pathlib.Path(
            "data",
            "assessments.phq9.transformed",
        ),
        dataframe_format_export(
            df_documents[
                (df_documents["_type"] == "assessmentLog")
                & (df_documents["assessmentId"] == "phq-9")
            ],
            drop_empty_columns=True,
        ),
    )

    # Formatted GAD-7 and PHQ-9 documents.
    drop_columns = [
        "_type",
        "_set_id",
        "patientSubmitted",
        "pointValues",
        "recordedDateTime",
        "scheduledAssessmentId",
        "totalScore",
    ]
    rename_columns = {
        "_id": "_docId",
        "assessmentLogId": "_assessmentLogId",
        "assessmentLogRecordedDate": "recordedDate",
        "assessmentLogScheduledAssessmentId": "todo_scheduledAssessmentId",
        "submittedByProviderId": "todo_submittedByProviderId",
        "assessmentLogSubmittedBy": "submittedBy",
    }
    sort_columns = [
        "recordId",
        "_patientId",
        "_docId",
        "_assessmentLogId",
        "_rev",
        "_created",
        "assessmentId",
        "recordedDate",
        "submittedBy",
        "todo_scheduledAssessmentId",
        "todo_submittedByProviderId",
        "gad7Anxious",
        "gad7ConstantWorrying",
        "gad7WorryingTooMuch",
        "gad7TroubleRelaxing",
        "gad7Restless",
        "gad7Irritable",
        "gad7Afraid",
        "gad7Score",
        "phq9Interest",
        "phq9Mood",
        "phq9Sleep",
        "phq9Energy",
        "phq9Appetite",
        "phq9Guilt",
        "phq9Concentrating",
        "phq9Motor",
        "phq9Suicide",
        "phq9Score",
        "comment",
    ]
    sort_rows_by_columns = [
        "recordId",
        "_patientId",
        "_assessmentLogId",
        "_rev",
    ]

    export_dataframe(
        pathlib.Path("assessments.gad7"),
        dataframe_format_export(
            df_documents[
                (df_documents["_type"] == "assessmentLog")
                & (df_documents["assessmentId"] == "gad-7")
            ],
            drop_empty_columns=True,
            drop_columns=drop_columns,
            rename_columns=rename_columns,
            sort_columns=sort_columns,
            sort_rows_by_columns=sort_rows_by_columns,
        ),
    )

    export_dataframe(
        pathlib.Path("assessments.phq9"),
        dataframe_format_export(
            df_documents[
                (df_documents["_type"] == "assessmentLog")
                & (df_documents["assessmentId"] == "phq-9")
            ],
            drop_empty_columns=True,
            drop_columns=drop_columns,
            rename_columns=rename_columns,
            sort_columns=sort_columns,
            sort_rows_by_columns=sort_rows_by_columns,
        ),
    )


export_analysis_assessments()

### Analysis: Mood Logs

In [ ]:
def export_analysis_mood_logs():
    # Documentation of this analysis.
    export_markdown(
        pathlib.Path("moodLogs"),
        documentation_as_markdown("Mood Logs"),
    )

    # Preliminary documents.
    export_dataframe(
        pathlib.Path(
            "data",
            "moodLogs.raw",
        ),
        dataframe_format_export(
            df_documents_raw[(df_documents_raw["_type"] == "moodLog")],
            drop_empty_columns=True,
        ),
    )

    export_dataframe(
        pathlib.Path(
            "data",
            "moodLogs.transformed",
        ),
        dataframe_format_export(
            df_documents[(df_documents_raw["_type"] == "moodLog")],
            drop_empty_columns=True,
        ),
    )

    # Formatted mood logs.
    drop_columns = [
        "_type",
        "_set_id",
        "recordedDateTime",
    ]
    rename_columns = {
        "_id": "_docId",
        "moodLogId": "_moodLogId",
    }
    sort_columns = [
        "recordId",
        "_patientId",
        "_docId",
        "_moodLogId",
        "_rev",
        "_created",
        "mood",
        "comment",
    ]
    sort_rows_by_columns = [
        "recordId",
        "_patientId",
        "_created",
    ]

    export_dataframe(
        pathlib.Path("moodLogs"),
        dataframe_format_export(
            df_documents[(df_documents["_type"] == "moodLog")],
            drop_empty_columns=True,
            drop_columns=drop_columns,
            rename_columns=rename_columns,
            sort_columns=sort_columns,
            sort_rows_by_columns=sort_rows_by_columns,
        ),
    )


export_analysis_mood_logs()

### Write Archive

In [ ]:
# The export is stored in a single zip file.
with open(
    pathlib.Path(
        archive_dir_path,
        "export_{}.zip".format(archive_suffix),
    ),
    mode="xb",
) as archive_file:
    with pyzipper.AESZipFile(
        archive_file,
        "w",
        compression=pyzipper.ZIP_LZMA,
        encryption=pyzipper.WZ_AES,
    ) as archive_zipfile:
        # Set the password
        archive_zipfile.setpassword(archive_password.encode("utf-8"))

        for file_current in export_file_list:
            if file_current.type in [
                ExportFileType.BYTES,
                ExportFileType.EXCEL,
            ]:
                archive_zipfile.writestr(
                    file_current.path.as_posix(), file_current.bytes
                )
            elif file_current.type in [
                ExportFileType.CSV,
                ExportFileType.MARKDOWN,
            ]:
                archive_zipfile.writestr(
                    file_current.path.as_posix(), file_current.text.encode("utf-8")
                )
            else:
                raise ValueError("Unknown ExportFileType")